In [118]:
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv("/kaggle/input/conductivity/train.csv")

# data Importing and Preprocessing

In [119]:
data.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.0
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.0
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.0
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.0


In [83]:
#unnamed_columns = data.columns[data.columns.str.contains('^Unnamed')]

#data = data.drop(columns=unnamed_columns)

In [85]:
#data = data.drop(['Date','Time'],axis=1)

In [104]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21263 entries, 0 to 21262
Data columns (total 82 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   number_of_elements               21263 non-null  int64  
 1   mean_atomic_mass                 21263 non-null  float64
 2   wtd_mean_atomic_mass             21263 non-null  float64
 3   gmean_atomic_mass                21263 non-null  float64
 4   wtd_gmean_atomic_mass            21263 non-null  float64
 5   entropy_atomic_mass              21263 non-null  float64
 6   wtd_entropy_atomic_mass          21263 non-null  float64
 7   range_atomic_mass                21263 non-null  float64
 8   wtd_range_atomic_mass            21263 non-null  float64
 9   std_atomic_mass                  21263 non-null  float64
 10  wtd_std_atomic_mass              21263 non-null  float64
 11  mean_fie                         21263 non-null  float64
 12  wtd_mean_fie      

# Normalizing data for better MSE

In [120]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Create an instance of the scaler
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

# Normalize the data using Min-Max scaling
normalized_data = minmax_scaler.fit_transform(data)

# Normalize the data using Standardization
standardized_data = standard_scaler.fit_transform(data)

In [122]:
X = standardized_data[:, :-1]  # Select all columns except the last column
y = standardized_data[:, -1]   # Select only the last column

# Splitting Of data

In [123]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# **Important Functions**

**Fitness**

In [124]:
def evaluate_fitness(chromosome, X, y):
    predictions = np.dot(X, chromosome)
    error = np.mean((predictions - y) ** 2)
    fitness = -error
    return fitness

**Cross-overs**

In [125]:
def single_point_crossover(parent1, parent2):
    # Randomly select crossover point
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    
    return child1, child2

def two_point_crossover(parent1, parent2):
    # Randomly select two crossover points
    crossover_point1 = np.random.randint(1, len(parent1)-1)
    crossover_point2 = np.random.randint(crossover_point1+1, len(parent1))

    child1 = np.concatenate((parent1[:crossover_point1], parent2[crossover_point1:crossover_point2], parent1[crossover_point2:]))
    child2 = np.concatenate((parent2[:crossover_point1], parent1[crossover_point1:crossover_point2], parent2[crossover_point2:]))
    
    return child1, child2

def uniform_crossover(parent1, parent2):
    mask = np.random.randint(2, size=len(parent1)).astype(bool)
    
    # Perform crossover
    child1 = parent1.copy()
    child2 = parent2.copy()
    child1[mask] = parent2[mask]
    child2[mask] = parent1[mask]
    
    return child1, child2


**Mutations**

In [126]:

def mutate(chromosome, mutation_rate):
    mutated_chromosome = chromosome.copy()
    
    for i in range(len(chromosome)):
        if np.random.uniform() < mutation_rate:
            mutated_chromosome[i] = 1 - mutated_chromosome[i]  # Flip the bit
            
    return mutated_chromosome

def perform_mutation(population, mutation_rate):
    mutated_population = []
    
    for chromosome in population:
        mutated_chromosome = mutate(chromosome, mutation_rate)
        mutated_population.append(mutated_chromosome)
        
    return mutated_population

**Selections**

In [127]:
def roulette_wheel_selection(population, fitness_values, num_selected):
    selected_population = []
    total_fitness = sum(fitness_values)
    
    for _ in range(num_selected):
        r = np.random.uniform(0, total_fitness)
        cumulative_fitness = 0
        
        for i, chromosome in enumerate(population):
            cumulative_fitness += fitness_values[i]
            
            if cumulative_fitness >= r:
                selected_population.append(chromosome)
                break
                
    return selected_population
def tournament_selection(population, fitness_values, num_selected, tournament_size):
    selected_population = []
    population_size = len(population)
    
    for _ in range(num_selected):
        participants_indices = np.random.choice(population_size, size=tournament_size, replace=False)
        participants_fitness = [fitness_values[i] for i in participants_indices]
        winner_index = participants_indices[np.argmax(participants_fitness)]
        selected_population.append(population[winner_index])
        
    return selected_population

def rank_based_selection(population, fitness_values, num_selected):
    selected_population = []
    population_size = len(population)
    
    ranks = np.argsort(fitness_values)
    selection_probabilities = np.linspace(0, 1, population_size)[::-1]  # Reverse probabilities
    
    for _ in range(num_selected):
        r = np.random.uniform(0, 1)
        cumulative_probability = 0
        
        for i, probability in zip(ranks, selection_probabilities):
            cumulative_probability += probability
            
            if cumulative_probability >= r:
                selected_population.append(population[i])
                break
                
    return selected_population

# **Genetic Algorithm**

In [145]:
import numpy as np
population_size = 200
chromosome_length = X_train.shape[1]

population = []
for _ in range(population_size):
    chromosome = np.random.uniform(low=-1, high=1, size=chromosome_length)
    population.append(chromosome)

mutation_rate = 0.2
crossover_type = 'two_point'
iteration = 1
#Main
while iteration<1000:
    fitness_values = []
    for chromosome in population:
        fitness = evaluate_fitness(chromosome, X_train, y_train)
        fitness_values.append(fitness)

    print(f"Generation: {iteration}")
    print("Fitness Values:", fitness_values)
    #print("Population:")
    #for chromosome in population:
       # print(chromosome)
    print()
    selected_population = tournament_selection(population, fitness_values, 10,10)
    new_population = []
    for i in range(0, len(selected_population) - 1, 2):
        parent1 = selected_population[i]
        parent2 = selected_population[i + 1]

        if crossover_type == 'single_point':
            child1, child2 = single_point_crossover(parent1, parent2)
        elif crossover_type == 'two_point':
            child1, child2 = two_point_crossover(parent1, parent2)
        elif crossover_type == 'uniform':
            child1, child2 = uniform_crossover(parent1, parent2)
        else:
            raise ValueError(f"Invalid crossover type: {crossover_type}")

        new_population.append(child1)
        new_population.append(child2)

    if len(selected_population) % 2 != 0:
        new_population.append(selected_population[-1])

    mutated_population = perform_mutation(new_population, mutation_rate)
    population = mutated_population

    iteration += 1

#---------------------------------------------------------------------------------#

if fitness_values:
    best_chromosome = population[np.argmax(fitness_values)]
else:
    best_chromosome = None

print("Best weights:", best_chromosome)
print("Fitness:", best_fitness)

Generation: 1
Fitness Values: [-34.55673483106541, -29.096851619766635, -27.74034594213432, -12.903046364156193, -63.05140641307411, -75.98641210811637, -16.825208798615442, -94.80362670252624, -30.392270494410095, -57.58428035998248, -12.24181060599711, -84.16349243707874, -19.044219317451336, -16.07859468519898, -45.38836465238413, -17.113762693049548, -8.480248691838064, -17.658043256468805, -35.97497931585294, -17.785548477670687, -109.61518295442296, -36.27436514156809, -18.3566764527486, -16.158524777027957, -18.58283658626285, -22.702720360050723, -27.784618077345563, -27.54376642755068, -19.03275521388071, -22.27784064265436, -14.23307353539908, -33.30717697079217, -65.17736458480623, -24.056196441343605, -7.620637305080436, -5.948755579090051, -30.97890298018194, -37.910148159848085, -14.12488825905192, -41.00609613471933, -21.855658238121638, -37.641758166210806, -17.63630232052819, -22.774387791802564, -22.51209137234023, -20.781354547916816, -26.38883427777028, -27.52479671

# **Weights Evaluation**

In [146]:
predictions = np.dot(X_test, best_chromosome)
mse = np.mean((predictions - y_test) ** 2)

print("Mean Squared Error:", mse)

Mean Squared Error: 93.00030345088953
